In [1]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt

In [2]:
import folium
from folium.plugins import HeatMap

In [3]:
dir_path = r'C:\Users\91989\Downloads\Dataset'
file = os.listdir(dir_path)
print("Total no. of files:", len(file))

Total no. of files: 10


In [4]:
with open(os.path.join(dir_path, file[0])) as f:
    json_object = json.load(f)

print(len(json_object))

27019


In [5]:

keys = list(json_object[0].keys())
print("The keys in data are:", keys)

The keys in data are: ['full_text', 'text_translation', 'created_at', 'screen_name', 'description', 'desc_translation', 'weekofyear', 'weekday', 'day', 'month', 'year', 'location', 'point_info', 'point', 'latitude', 'longitude', 'altitude', 'province', 'hisco_standard', 'hisco_code', 'industry', 'sentiment_pattern', 'subjective_pattern']


In [6]:
df = pd.DataFrame(data = {}, columns = keys)

In [7]:

df.columns[df.columns.duplicated()].to_list()

[]

In [10]:

for i in range(len(file)):
  with open(os.path.join(dir_path, file[i])) as f:
    json_obj = json.load(f)
    df = pd.concat([df, pd.DataFrame(json_obj)], ignore_index=True)

print(df.shape)

C:\Users\91989\AppData\Local\Temp\ipykernel_12040\741337765.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(json_obj)], ignore_index=True)


(271342, 23)


In [11]:

df_1 = df.copy()

In [30]:

df_1.head()

,full_text,text_translation,created_at,screen_name,description,desc_translation,weekofyear,weekday,day,month,...,point,latitude,longitude,altitude,province,hisco_standard,hisco_code,industry,sentiment_pattern,subjective_pattern
0,"@pflegearzt @Friedelkorn @LAguja44 Pardon, wol...",@pflegearzt @Friedelkorn @ LAguja44 Pardon wol...,1583756789000,TheoRettich,"I ❤️science, therefore a Commie. ☭ FALGSC: P...","I ❤️science, Therefore a Commie. ☭ FALGSC: Par...",11,0,9,3,...,"(52.5001698, 5.7480821, 0.0)",52.50017,5.748082,0.0,Flevoland,None,None,False,0.0,0.0
1,RT @grantshapps: Aviation demand is reduced du...,RT @grantshapps: Aviation demand is reduced du...,1583756794000,davidiwanow,I tweet a lot but love to engage & converse. P...,I tweet a lot but love to engage and converse....,11,0,9,3,...,"(52.3727598, 4.8936041, 0.0)",52.37276,4.893604,0.0,Noord-Holland,None,None,False,0.0,0.0
2,RT @DDStandaard: De droom van D66 wordt werkel...,RT @DDStandaard: The D66 dream come true: COVI...,1583756797000,EricL65,None,None,11,0,9,3,...,None,NaN,NaN,0.0,False,None,None,False,0.0,0.0
3,RT @DDStandaard: De droom van D66 wordt werkel...,RT @DDStandaard: The D66 dream come true: COVI...,1583756797000,EricL65,None,None,11,0,9,3,...,None,NaN,NaN,0.0,False,None,None,False,0.0,0.0
4,De droom van D66 wordt werkelijkheid: COVID-19...,The D66 dream becomes reality: COVID-19 super ...,1583756807000,EhrErwin,Budget-Life Coach. Time management Coaching. b...,Budget-Life Coach. Time management coaching. h...,11,0,9,3,...,"(52.3727598, 4.8936041, 0.0)",52.37276,4.893604,0.0,Noord-Holland,None,None,False,0.0,0.0


In [40]:

df_map = df_1[['created_at', 'location', 'latitude', 'longitude', 'altitude', 'province']]
df_map.head()
print(df_map.shape)

(271342, 6)


In [41]:
df_map.dropna(subset=['longitude', 'latitude'], inplace=True)

C:\Users\91989\AppData\Local\Temp\ipykernel_12040\2587569966.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_map.dropna(subset=['longitude', 'latitude'], inplace=True)


In [42]:
df_map = df_map[(df_map['latitude'] >= -90) & (df_map['latitude'] <= 90)]
df_map = df_map[(df_map['longitude'] >= -180) & (df_map['longitude'] <= 180)]

In [43]:
df_map['latitude'] = df_map['latitude'].round(4)
df_map['longitude'] = df_map['longitude'].round(4)

In [44]:

df_map.shape

(134445, 6)

In [45]:
df_group = df_map.drop(['location' ,'altitude','province'], axis='columns')

In [46]:
map_center = [df_group['latitude'].mean(), df_group['longitude'].mean()]  # note: folium uses [lat, lon] order
heat_map = folium.Map(location=map_center, zoom_start=4)

In [47]:
head_data = df_group[['latitude', 'longitude']].values.tolist()
HeatMap(head_data, radius=10, blur=15, max_zoom=6).add_to(heat_map)

In [48]:

image = "dutch_heatmap.html"
heat_map.save(image)

In [49]:

print(df['full_text'].head())

0    @pflegearzt @Friedelkorn @LAguja44 Pardon, wol...
1    RT @grantshapps: Aviation demand is reduced du...
2    RT @DDStandaard: De droom van D66 wordt werkel...
3    RT @DDStandaard: De droom van D66 wordt werkel...
4    De droom van D66 wordt werkelijkheid: COVID-19...
Name: full_text, dtype: object


In [50]:

print(df['text_translation'].head())

0    @pflegearzt @Friedelkorn @ LAguja44 Pardon wol...
1    RT @grantshapps: Aviation demand is reduced du...
2    RT @DDStandaard: The D66 dream come true: COVI...
3    RT @DDStandaard: The D66 dream come true: COVI...
4    The D66 dream becomes reality: COVID-19 super ...
Name: text_translation, dtype: object


In [51]:

df_comments = df[['text_translation']]

In [52]:
import nltk

nltk.download([
"names",
"stopwords",
"state_union",
"twitter_samples",
"movie_reviews",
"averaged_perceptron_tagger",
"vader_lexicon",
"punkt",
])

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\91989\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\names.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91989\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package state_union to
[nltk_data]     C:\Users\91989\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\state_union.zip.
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\91989\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\twitter_samples.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\91989\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\91989\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_

True

In [53]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [54]:

sia = SentimentIntensityAnalyzer()

In [55]:

df_comments.head()

,text_translation
0,@pflegearzt @Friedelkorn @ LAguja44 Pardon wol...
1,RT @grantshapps: Aviation demand is reduced du...
2,RT @DDStandaard: The D66 dream come true: COVI...
3,RT @DDStandaard: The D66 dream come true: COVI...
4,The D66 dream becomes reality: COVID-19 super ...


In [56]:

df_comments.dropna(inplace=True)

C:\Users\91989\AppData\Local\Temp\ipykernel_12040\3397017784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comments.dropna(inplace=True)


In [57]:

def senti_analysis(text):
    scores = sia.polarity_scores(text)
    return scores['compound']

In [58]:
df_comments['sentiment_scores'] = df_comments['text_translation'].apply(senti_analysis)

C:\Users\91989\AppData\Local\Temp\ipykernel_12040\934117715.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comments['sentiment_scores'] = df_comments['text_translation'].apply(senti_analysis)


In [60]:
def sentiments(scores):
    if scores > 0.05:
        return "positive"

    elif scores <= -0.05:
        return "negative"

    else:
        return "neutral"

In [61]:
df_comments['sentiment_label'] = df_comments['sentiment_scores'].apply(sentiments)

C:\Users\91989\AppData\Local\Temp\ipykernel_12040\626586181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comments['sentiment_label'] = df_comments['sentiment_scores'].apply(sentiments)


In [62]:
df_comments.to_csv("sentimentAnalysis.csv", index = False)